In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from model import InvertedPendulum, massSpringDamper
import matplotlib.animation as animation
from functools import partial

In [2]:
class PID_controller():
    def __init__(self, kp, ki, kd):
        self.kp = kp
        self.ki = ki
        self.kd = kd

    def action(self, error, errorDot, errorInt):
        return self.kp*error + self.ki*errorInt + self.kd*errorDot
    
class stateFeedback_controller():
    def __init__(self, gain):
        self.gain = gain
    
    def action(self, state):
        return np.dot(self.gain, state)
    
def update(frame, theta_arr, pendCen, pendLen, line): #function for creating animation
    theta = theta_arr[frame]
    x = [pendCen[0], pendCen[0] - pendLen*np.sin(theta)]
    z = [pendCen[1], pendCen[1] - pendLen*np.cos(theta)]
    line.set_xdata(x)
    line.set_ydata(z)
    return line

In [3]:
def main():
    m_ivp = 1
    k_ivp = 1
    l_ivp = 1
    g = 9.8
    procNoiseCov = 2
    mesNoiseCov = 0.02
    isDisturb_ivp = 1

    inverted_pendulum = InvertedPendulum(m_ivp, l_ivp, g, k_ivp, isDisturb_ivp, procNoiseCov, mesNoiseCov)

    pid_controller = PID_controller(10,0,0)

    sim_time = 0.15

    sim_dt = 0.05
    
    theta_0 = np.pi/2
    theta_T = np.pi
    omega_0, omega_T = 0,0

    #implementing PID
    error_p = 0
    theta = theta_0
    omega = omega_0
    theta_array = [theta]
    omega_array = [omega]
    error_int = 0
    time_array = [0]
    u_pid_array = []
    error_array = []
    for time in np.arange(0, sim_time, sim_dt):
        error = theta_T - theta
        error_dot = (error - error_p)/sim_dt
        error_int += error*sim_dt
        u_pid = pid_controller.action(error, error_dot, error_int)
        theta_next, omega_next = inverted_pendulum.nextState([theta, omega], u_pid, sim_dt)
        theta, omega = theta_next, omega_next
        theta_array.append(theta)
        omega_array.append(omega)
        time_array.append(time+sim_dt)
        u_pid_array.append(u_pid)
        error_array.append(error)

    # plt.figure(1)
    # ax1 = plt.subplot(2,2,1)
    # ax1.plot(time_array, theta_array)
    # ax1.title.set_text('theta')
    # ax2 = plt.subplot(2,2,2)
    # ax2.plot(time_array, omega_array)
    # ax2.title.set_text('omega')
    # ax3 = plt.subplot(2,2,3)
    # ax3.plot(time_array[:-1], u_pid_array)
    # ax3.title.set_text('control')
    # ax4 = plt.subplot(2,2,4)
    # ax4.plot(time_array[:-1], error_array)
    # ax4.title.set_text('error')

    #implementing state feedback assuming full state is available

    x_T = np.array([[theta_T],[omega_T]])
    x_0 = np.array([[theta_0],[omega_0]])
    x_hat = np.copy(x_0)
    x_dim = len(x_0)
    
    #x_array_sf = [x_0]
    x_array_sf = np.zeros((2, int(sim_time/sim_dt+1)))
    x_array_sf[:,0] = x_0.reshape(x_dim,)

    x_hat_array = np.zeros((2, int(sim_time/sim_dt+1)))
    x_hat_array[:,0] = x_hat.reshape(x_dim,)
    innov_array = []
    kf_cov_array = []

    omega_array_sf = [omega_0]
    theta_array_sf = [theta_0]
    u_sf_array = []
    error_sf_array = []

    x = np.copy(x_0)
    
    p_hat = np.identity(2)*0
    B_mat = inverted_pendulum.BMatt()
    C_mat = inverted_pendulum.CMatt
    D_mat = inverted_pendulum.DMatt
    for time in np.arange(0, sim_time, sim_dt):
        A_mat = inverted_pendulum.AMatt(x[0,0])

        poles = np.array([-10,-25])
        K_mat = sc.signal.place_poles(A_mat, B_mat, poles).gain_matrix
        u_ss = inverted_pendulum.m*g*inverted_pendulum.l/2*np.cos(x[0,0])*(x_T[0,0] - x_hat[0,0])
        #print('u_ss - ', u_ss, 'K_mat - ',  K_mat, 'diff - ', x_T - x)
        u_tot = (u_ss + np.dot(K_mat,(x_T - x_hat))[0,0])
        #u_tot = (time < 1)*u_ss + (time >=1 and time < 4)*3*u_ss + (time >=4)*u_ss
        print('updating actual state')
        #Look into how odeint works
        disturb = np.random.normal(0, procNoiseCov, 1)[0]
        theta_next, omega_next = inverted_pendulum.nextState([x[0,0], x[1,0]], u_tot, sim_dt, disturb)

        if theta_next > 2*np.pi:
            #print('it happened')
            theta_next = theta_next % (2*np.pi)
        elif theta_next < 0:
            #print('it happened1')
            theta_next = 2*np.pi - ((-theta_next) % (2*np.pi))
        x = np.array([[theta_next],[omega_next]])
        x_array_sf[:, int(round(1+(time/sim_dt)))] = x.reshape(x_dim,)
    
        y_meas = inverted_pendulum.measurement(x)
        d_syst = sc.signal.cont2discrete((A_mat, B_mat, C_mat, D_mat), sim_dt)
        
        Phi, Gamma, C_d = d_syst[0], d_syst[1], d_syst[2]
        
        Gamma_d = np.copy(Gamma)
        #x_pred = np.dot(Phi, x_hat) + np.dot(Gamma, u_tot) - np.array([[0], [1.5*g/l_ivp*np.sin(x_hat[0,0])*sim_dt]]).reshape(2,1)
        print('updating estimate')
        theta_pred, omega_pred = inverted_pendulum.nextState([x_hat[0,0], x_hat[1,0]],u_tot,sim_dt)
        x_pred = np.array([[theta_pred],[omega_pred]])
        # if time == 0.5:
        #     print(x_pred, np.array([[0], [1.5*g/l_ivp*np.sin(x_hat[0,0])*sim_dt]]).reshape(2,1))
        p_pred = Phi @ p_hat @ np.transpose(Phi) + Gamma_d @ np.transpose(Gamma_d)*procNoiseCov
        # print('p_hat', p_hat)
        # print('p_pred', p_pred)
        print('Gamma_d', Gamma_d)
        print('second part in p_pred', Gamma_d @ np.transpose(Gamma_d)*procNoiseCov)
        kalman_gain = (p_pred @ np.transpose(C_d)) @ np.linalg.inv(C_d @ p_pred @ np.transpose(C_d) + mesNoiseCov)
        #print(kalman_gain)
        innov = y_meas - (C_d @ x_pred)
        #print(y_meas, C_d @x_pred, innov)
        x_hat = x_pred + kalman_gain @ innov
        if x_hat[0,0] > 2*np.pi:
            x_hat[0,0] = x_hat[0,0] % (2*np.pi)
        elif x_hat[0,0] < 0:
            x_hat[0,0] = 2*np.pi - ((-x_hat[0,0]) % (2*np.pi))
            
        #p_hat = p_pred - kalman_gain @ C_d @ p_pred
        p_hat = (np.identity(2) - kalman_gain @ C_d) @ p_pred
        
        x_hat_array[:, 1+int(round((time/sim_dt)))] = x_hat.reshape(x_dim,)
        kf_cov_array.append(np.trace(p_hat))
        innov_array.append(innov[0,0])
        theta_array_sf.append(theta_next)
        omega_array_sf.append(omega_next)
        u_sf_array.append(u_tot)
        error_sf_array.append(theta_T - theta_next)
    #print(x_array_sf[0,-20:])
    #print(innov_array)
    est_error_array = x_array_sf - x_hat_array
    print(np.trace(p_hat))
    plt.figure(2)
    ax1 = plt.subplot(2,2,1)
    ax1.plot(time_array, x_array_sf[0,:])
    ax1.title.set_text('theta')
    ax2 = plt.subplot(2,2,2)
    ax2.plot(time_array,  omega_array_sf)
    ax2.title.set_text('omega')
    ax3 = plt.subplot(2,2,3)
    ax3.plot(time_array[:-1], u_sf_array)
    ax3.title.set_text('control')
    ax4 = plt.subplot(2,2,4)
    ax4.plot(time_array[:-1], error_sf_array)
    ax4.title.set_text('error')

    plt.figure(3)
    ax1 = plt.subplot(4,1,1)
    ax1.plot(time_array, x_array_sf[0,:])
    ax1.plot(time_array, x_hat_array[0,:])
    ax1.legend(['actual', 'estimate'])
    
    ax2 = plt.subplot(4,1,2)
    ax2.plot(time_array, x_array_sf[1,:])
    ax2.plot(time_array, x_hat_array[1,:])
    ax2.legend(['actual', 'estimate'])

    ax3 = plt.subplot(4,1,3)
    ax3.plot(time_array[1:len(time_array)], innov_array)

    ax4 = plt.subplot(4,1,4)
    ax4.plot(time_array[1:len(time_array)], kf_cov_array)

    plt.figure(4)
    ax1 = plt.subplot(2,1,1)
    ax1.plot(time_array, est_error_array[0,:])

    ax2 = plt.subplot(2,1,2)
    ax2.plot(time_array, est_error_array[1,:])

    #creating animation
    fig, ax = plt.subplots()
    pendCent = [2,2]
    offset = [-l_ivp*np.sin(theta_0), -l_ivp*np.cos(theta_0)]
    pendEnd = pendCent + offset

    line = ax.plot([pendCent[0], pendEnd[0]], [pendCent[1], pendEnd[1]], 'k-', linewidth=3)[0]
    ax.plot(pendCent[0], pendCent[1], 'kx')
    ax.plot(pendEnd[0], pendEnd[1], 'ro')
    #print(pendEnd)
    ax.set(xlim = [0,4], ylim = [0,4], xlabel = "x", ylabel = "z")
    #ax.set_aspect('equal')
    plt.gca().set_aspect('equal')
    plt.axis('off')
    ani = animation.FuncAnimation(fig=fig, func=partial(update, theta_arr = theta_array_sf, pendCen = pendCent, pendLen = l_ivp, line = line), frames=int(sim_time/sim_dt), interval = int(sim_dt*1000))
    #writervideo = animation.FFMpegWriter(fps=60) 
    ani.save('Invpend_SF.gif')#, writer=writervideo) 
    plt.close

In [4]:
if __name__ == '__main__':
    main()
    

0.9926567967361166
-3.242349913836697
2.8091626837972807
1.3366375605156682
-0.2069187307431452
-4.131955911233877
2.135068028843937
2.3382853869555777
0.02844261354527282
-0.826787331283877
-3.3458284047346836
-2.2469955815579623
2.0557794631435873
3.47819489544903
0.9637669084214496
0.9476772156826713
0.9069186879665626
0.7553484225274003
-6.262942249247585
-0.28718874688900015
1.3087570028459143
2.1898321738542275
-0.4300813599690656
-1.9578208128601766
-1.0541368326070182
-1.5220337540240427
-1.82053132982255
-0.6221070921442661
-2.6999874497388068
2.7748568704654555
0.6923158750799409
-0.886772148409104
-1.8446942946636309
0.5626504575221899
0.33984538183933877
0.6919694987498153
1.848128541697959
-0.1557305847059303
0.3530446188248316
-4.11736171871573
0.06946997480339298
-3.454761866057775
-0.45848951074108835
-0.43178292371251764
0.6949274464956613
1.8971012677389336
-2.9498017833612065
0.11517803686821092
-3.1597129117453333
-1.1631822087748136
-1.3450864465129944
2.2656513692

C:\Users\karan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\integrate\_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


updating actual state
0.4242257390578578
-2.1354417494486015
2.2938626699972593
-2.865318419613565
2.70633772094042
2.4554044426043884
-0.3165891489128494
1.186132996058946
3.696803640152072
-0.03477673481014593
-0.9766888203544009
-0.6534857872591348
-0.9143180225532992
-2.1610500912302477
0.22169656451191233
-3.4788708619983497
1.6158121826172271
3.0126004291971133
-1.651656817110525
0.11737134331171425
-0.8057716751893894
1.6911132166995566
3.285830076089331
-2.031962677913492
-3.6720530202585078
1.7223264214044018
1.8951435668681202
0.33986698043494196
0.1276245742375246
-4.104918338780195
-0.38394441364080734
-1.5311996180252818
2.3369796004203787
0.6045226751035466
-1.2583857968160308
-1.653198451139097
-2.486784797295151
-2.456708938303702
-0.5207691236304353
1.651568345162806
0.11819401505128314
-2.339510813186096
2.698376423701572
1.6278652783889043
-0.5427709014503769
0.4950598279496167
1.836680738157718
2.287880469310688
2.167832884289649
-2.485881896893812
-1.52988441787350

IndexError: index 3 is out of bounds for axis 1 with size 3

In [77]:
a = 8.999
print(int(a), int(round(a)))
print(np.random.normal(0,1,1))


8 9
[1.38905969]


In [16]:
help('np.hist')

No Python documentation found for 'np.hist'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.

